# pre_processing

In [20]:
import pandas as pd
import numpy as np

In [21]:
import nbimporter
import feature_generation

In [22]:
features = feature_generation.get_features()
features_types = feature_generation.get_features_types()

In [23]:
def load_appended_dataset():
    """Carga ambos dataframes (train y test) y devuelve un solo dataframe que es el resultado de realizar un
    append entre ambos. El dataset de test esta al final."""

    train = pd.read_csv('./data/train.csv', index_col='id', parse_dates=['fecha'])
    test = pd.read_csv('./data/test.csv', index_col='id', parse_dates=['fecha'])
    df = train.append(test, sort=False)
    df = df[['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
         'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
         'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio']]
    
    whitelist_tipodepropiedad = df['tipodepropiedad'].value_counts().index.to_list()[:9]
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: x if x in whitelist_tipodepropiedad else 'Otro') 
    
    #whitelist_ciudad = df['ciudad'].value_counts().index.to_list()[:80]
    #df['ciudad'] = df['ciudad'].apply(lambda x: x if x in whitelist_ciudad else 'Otro')
    
    categoricas = ['tipodepropiedad', 'ciudad', 'provincia']
    df[categoricas] = df[categoricas].fillna('unknown')
    df[categoricas] = df[categoricas].astype('category')
    enteras = ['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']
    df[enteras] = df[enteras].astype('int8')
    
    return df

In [24]:
def separate_train_test(df):
    """Separa df compuesto por sus primeras 240.000 filas de train.csv y sus 60.000 ultimas filas de test
    en los dos dataframes correspondientes."""
    
    train = df.head(240000)
    test = df.tail(60000).drop('precio', axis=1)
    return train, test

In [25]:
def load_datasets():
    """Carga ambos dataframes (train y test) y devuelve dos datasets en el formato:
    train_df, test_df"""
    
    df = load_appended_dataset()
    return separate_train_test(df)

In [26]:
def load_featured_appended_dataset():
    """Carga el .csv con los features generados, y lo formatea para ahorrar memoria. Devuelve un unico
    dataframe cuyas primeras 240.000 filas son de train.csv y sus ultimas 60.000 de test."""
    features_types = feature_generation.get_features_types()
    
    df = pd.read_csv('./data/full_featured.csv', index_col='id')
    df[features_types['bernoulli']] = df[features_types['bernoulli']].astype('uint8')
    df[features_types['float']] = df[features_types['float']].astype('float')
    df[features_types['uint8']] = df[features_types['uint8']].astype('uint8')
    df[features_types['uint16']] = df[features_types['uint16']].astype('uint16')
    df[features_types['uint32']] = df[features_types['uint32']].astype('uint32')
    
    return df

In [27]:
def load_featured_datasets():
    """Devuelve dos datasets en el formato:
    train_df, test_df
    Para los datasets con las features generadas."""
    
    df = load_featured_appended_dataset()
    return separate_train_test(df)